<p align="center">
  <img src="../data/uach.png" alt="UACh" width="180">
</p>

# Hito 2 — Prototipo de Convolución

Proyecto ACUS220 · Acústica Computacional con Python

Este notebook implementa un prototipo funcional para auralización por convolución con una librería de presets de respuestas al impulso (IR). Permite generar una señal de entrada (o cargar una), seleccionar una IR (salas y un modelo escalonado tipo ‘Quetzal’), aplicar filtros simples y escuchar/visualizar el resultado, además de exportar a WAV.

**Integrantes:** Carlos Duarte, Fernando Castillo, Vicente Alves, Antonio Duque

## 1. Introducción

### 1.1 Problema

### 1.2 Herramienta de convolución

$ y(t) = x(t) * h(t) $

### 1.3 Objetivos de este análisis

## 2. Análisis preliminar

### 2.1 Generación de señales de prueba


In [ ]:
## Cargar por ejemplo un audio de un aplauso, graficar su forma de onda y espectrograma

### 2.2 Características de la señal

In [ ]:
## analisis: duración, frecuencia, espectro de magnitud, etc......

## 3. Respuesta al impulso (IR)

### 3.1 ¿Qué es IR?

### 3.2 ¿Qué es la convolución?


## Qué hace este prototipo
- Entradas: aplauso sintético, barrido senoidal o cargar WAV.
- Presets de IR: salas (pequeña/mediana/gran sala) y un modelo escalonado ‘Pirámide (Quetzal)’.
- Procesamiento: convolución FFT + normalización opcional + filtro simple (lowpass/highpass/bandpass).
- Salida: reproductor de audio, forma de onda, espectrograma y opción de exportar WAV a `hito2/data/outputs/`.

In [ ]:
import os, time, math, textwrap
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import soundfile as sf
import ipywidgets as widgets
from IPython.display import Audio, display, clear_output
import pyroomacoustics as pra

plt.rcParams['figure.figsize'] = (10, 4)
DATA_DIR = Path('data')
IN_DIR = DATA_DIR / 'inputs'
OUT_DIR = DATA_DIR / 'outputs'
IN_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)
print('Entorno listo.')


Entorno listo.


In [ ]:
# Utilidades de audio y visualización
def normalize_audio(x, peak=0.98):
    x = np.asarray(x, dtype=float)
    m = np.max(np.abs(x)) + 1e-12
    return (x / m) * peak

def resample_if_needed(x, sr, target_sr):
    if target_sr is None or sr == target_sr:
        return x, sr
    g = math.gcd(int(sr), int(target_sr))
    up = target_sr // g
    down = sr // g
    xr = signal.resample_poly(x, up, down)
    return xr.astype(float), target_sr

def load_audio(path, target_sr=None, mono=True):
    x, sr = sf.read(path, always_2d=False)
    if x.ndim > 1 and mono:
        x = x.mean(axis=1)
    x, sr = resample_if_needed(x, sr, target_sr)
    return x.astype(float), sr

def save_audio(path, x, sr):
    sf.write(str(path), x, int(sr))

def plot_waveform(x, sr, title='Señal'):
    t = np.arange(len(x)) / sr
    plt.figure()
    plt.plot(t, x, lw=0.9)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_spectrogram(x, sr, title='Espectrograma'):
    f, t, Sxx = signal.spectrogram(x, fs=sr, nperseg=1024, noverlap=512, scaling='spectrum')
    Sxx_db = 10 * np.log10(Sxx + 1e-12)
    plt.figure()
    plt.pcolormesh(t, f, Sxx_db, shading='auto', cmap='magma')
    plt.ylabel('Frecuencia [Hz]')
    plt.xlabel('Tiempo [s]')
    plt.ylim(0, sr/2)
    plt.title(title)
    plt.colorbar(label='dB')
    plt.tight_layout()
    plt.show()

def play(x, sr):
    display(Audio(x, rate=sr))

def make_synthetic_clap(sr=48000, duration_s=0.12, decay=3.5, noise_level=0.3):
    n = int(sr * duration_s)
    x = np.zeros(n)
    x[0] = 1.0  # impulso inicial
    rn = np.random.randn(n)
    env = np.exp(-np.linspace(0, decay, n))
    x += noise_level * rn * env
    return normalize_audio(x)

def make_sine_sweep(sr=48000, dur_s=2.0, f0=20.0, f1=12000.0):
    t = np.linspace(0, dur_s, int(sr*dur_s), endpoint=False)
    x = signal.chirp(t, f0=f0, f1=f1, t1=dur_s, method='logarithmic')
    win = signal.windows.tukey(len(x), alpha=0.1)
    return normalize_audio(x*win)

def butter_filter(x, sr, ftype='none', f_lo=200.0, f_hi=3000.0, order=4):
    x = np.asarray(x, dtype=float)
    nyq = 0.5 * sr
    f_lo = max(10.0, min(f_lo, nyq*0.99))
    f_hi = max(20.0, min(f_hi, nyq*0.99))
    if ftype == 'none':
        return x
    if ftype == 'lowpass':
        wn = min(f_hi/nyq, 0.999)
        b, a = signal.butter(order, wn, btype='low', output='ba')
    elif ftype == 'highpass':
        wn = max(f_lo/nyq, 1e-4)
        b, a = signal.butter(order, wn, btype='high', output='ba')
    elif ftype == 'bandpass':
        lo = max(1e-4, min(f_lo/nyq, 0.99))
        hi = max(lo*1.01, min(f_hi/nyq, 0.999))
        b, a = signal.butter(order, [lo, hi], btype='band', output='ba')
    else:
        return x
    return signal.lfilter(b, a, x)

def apply_ir(x, ir, normalize=True):
    y = signal.fftconvolve(x, ir, mode='full')
    return normalize_audio(y) if normalize else y


In [ ]:
# Generación de IRs: sala (Pyroomacoustics) y estructura escalonada (‘Quetzal’)
def pad_to_length(x, n):
    if len(x) >= n:
        return x[:n]
    out = np.zeros(n)
    out[:len(x)] = x
    return out

def simulate_room_ir(
    sr=48000,
    room_dim=(8.0, 5.0, 3.0),
    absorption=0.2,
    max_order=10,
    src_pos=(2.0, 2.0, 1.5),
    mic_pos=(6.0, 3.0, 1.5),
    ir_length_s=1.2,
):
    materials = pra.Material(absorption)
    room = pra.ShoeBox(room_dim, fs=sr, materials=materials, max_order=max_order)
    room.add_source(src_pos)
    mic_locs = np.array(mic_pos).reshape(-1, 1)
    room.add_microphone_array(pra.MicrophoneArray(mic_locs, room.fs))
    room.compute_rir()
    ir = np.asarray(room.rir[0][0], dtype=float)
    N = int(sr * ir_length_s)
    ir = pad_to_length(ir, N)
    return normalize_audio(ir)

def stepped_structure_ir(
    sr=48000,
    base_delay_ms=6.0,   # primer retardo
    step_delta_ms=1.2,   # incremento de retardo por eco
    accel_ms=-0.03,      # curvatura temporal (chirp en tiempos)
    num_steps=45,        # cantidad de ecos
    decay=0.92,          # atenuación por paso
    ir_length_s=1.4,
):
    delays_ms = []
    for n in range(num_steps):
        t = base_delay_ms + n * step_delta_ms + 0.5 * accel_ms * n * (n - 1)
        if t >= 0:
            delays_ms.append(t)
    if not delays_ms:
        return np.array([1.0])
    max_delay_ms = max(delays_ms)
    N = int(sr * ir_length_s)
    L = max(N, int(sr * (max_delay_ms / 1000.0)) + sr // 10)
    ir = np.zeros(L)
    for i, t_ms in enumerate(delays_ms):
        idx = int(round(sr * (t_ms / 1000.0)))
        if idx < L:
            ir[idx] += decay ** i
    ir[0] += 0.05
    ir = ir[:N] if N < len(ir) else ir
    return normalize_audio(ir)

def get_preset_ir(preset, sr):
    preset = str(preset).lower().strip()
    if preset.startswith('sala pequeña'):
        ir = simulate_room_ir(sr=sr, room_dim=(4.0, 3.0, 2.5), absorption=0.45, max_order=8,
                            src_pos=(1.2,1.0,1.2), mic_pos=(3.2,1.8,1.2), ir_length_s=1.0)
        label = 'Sala pequeña'
    elif preset.startswith('sala mediana'):
        ir = simulate_room_ir(sr=sr, room_dim=(8.0, 5.5, 3.2), absorption=0.25, max_order=12,
                            src_pos=(2.0,2.3,1.5), mic_pos=(6.0,3.1,1.5), ir_length_s=1.2)
        label = 'Sala mediana'
    elif 'hall' in preset or 'grande' in preset:
        ir = simulate_room_ir(sr=sr, room_dim=(16.0, 11.0, 6.0), absorption=0.12, max_order=15,
                            src_pos=(4.0,3.0,2.0), mic_pos=(12.0,7.0,2.5), ir_length_s=1.6)
        label = 'Sala grande / Hall'
    else:
        ir = stepped_structure_ir(sr=sr)
        label = 'Pirámide (Quetzal)'
    return ir, label


In [ ]:
# Interfaz interactiva con ipywidgets
# Controles
sr_dd = widgets.Dropdown(options=[48000, 44100], value=48000, description='Frecuencia de muestreo')
in_src = widgets.Dropdown(
    options=['Aplauso (sintético)', 'Barrido senoidal (20 Hz–12 kHz)', 'Cargar WAV (carpeta inputs)'],
    value='Aplauso (sintético)', description='Entrada'
)
wav_path = widgets.Text(value=str(IN_DIR / 'mi_audio.wav'), description='Ruta WAV', layout=widgets.Layout(width='50%'))

preset_dd = widgets.Dropdown(
    options=['Sala pequeña', 'Sala mediana', 'Sala grande / Hall', 'Pirámide (Quetzal) 🌪️'],
    value='Sala mediana', description='Preajuste de IR'
)

# Filtro de coloración (post-EQ)
eq_dd = widgets.Dropdown(options=['Ninguno', 'Pasa-bajos', 'Pasa-altos', 'Pasa-banda'], value='Ninguno', description='Filtro (posterior)')
f_lo = widgets.FloatSlider(value=200.0, min=20.0, max=5000.0, step=10.0, description='Frecuencia baja [Hz]')
f_hi = widgets.FloatSlider(value=3000.0, min=200.0, max=18000.0, step=10.0, description='Frecuencia alta [Hz]')
normalize_ck = widgets.Checkbox(value=True, description='Normalizar')

render_btn = widgets.Button(description='Procesar', button_style='primary')
save_btn = widgets.Button(description='Exportar WAV', button_style='')
out = widgets.Output()

state = {'y': None, 'sr': None, 'label': None, 'busy': False}

def _slug(s):
    return ''.join(ch for ch in s.lower() if ch.isalnum() or ch in ('-', '_')).replace(' ', '_')

def do_render(_=None):
    if state.get('busy'):
        return
    state['busy'] = True
    render_btn.disabled = True
    try:
        sr = int(sr_dd.value)
        # Entrada
        if in_src.value.startswith('Aplauso'):
            x = make_synthetic_clap(sr=sr)
        elif in_src.value.startswith('Barrido senoidal'):
            x = make_sine_sweep(sr=sr)
        else:
            x, _sr = load_audio(wav_path.value, target_sr=sr)
        # IR
        ir, label = get_preset_ir(preset_dd.value, sr)
        # Convolución
        y = apply_ir(x, ir, normalize=normalize_ck.value)
        # Post-EQ
        eq_mode = eq_dd.value.lower()
        if eq_mode != 'ninguno':
            mode_map = {'pasa-bajos': 'lowpass', 'pasa-altos': 'highpass', 'pasa-banda': 'bandpass'}
            y = butter_filter(y, sr, ftype=mode_map.get(eq_mode, 'none'), f_lo=f_lo.value, f_hi=f_hi.value, order=4)
            if normalize_ck.value:
                y = normalize_audio(y)
        # Estado para exportar
        state['y'] = y
        state['sr'] = sr
        state['label'] = label
        # Mostrar
        with out:
            clear_output(wait=True)
            print(f'Entrada: {in_src.value} | IR: {label} | Frecuencia de muestreo: {sr} Hz')
            print('— Señal de entrada')
            play(x, sr)
            print('— Respuesta al impulso (escuchar a bajo volumen)')
            play(ir, sr)
            print('— Señal de salida (convolución)')
            play(y, sr)
            plot_waveform(y, sr, title='Salida — forma de onda')
            plot_spectrogram(y, sr, title='Salida — espectrograma')
    finally:
        state['busy'] = False
        render_btn.disabled = False

def do_save(_=None):
    y = state.get('y'); sr = state.get('sr'); label = state.get('label') or 'out'
    if y is None or sr is None:
        with out: print('No hay resultado para exportar. Pulsa Procesar primero.')
        return
    fname = f'out_{_slug(label)}_{int(time.time())}.wav'
    fpath = OUT_DIR / fname
    save_audio(fpath, y, sr)
    with out: print('Exportado:', fpath)

# Evitar callbacks duplicados si se re-ejecuta la celda
try:
    render_btn._click_handlers.callbacks.clear()
    save_btn._click_handlers.callbacks.clear()
except Exception:
    pass
render_btn.on_click(do_render)
save_btn.on_click(do_save)

ui_top = widgets.HBox([sr_dd, in_src])
ui_path = widgets.HBox([wav_path])
ui_preset = widgets.HBox([preset_dd])
ui_eq = widgets.HBox([eq_dd, f_lo, f_hi, normalize_ck])
ui_btns = widgets.HBox([render_btn, save_btn])
ui = widgets.VBox([ui_top, ui_path, ui_preset, ui_eq, ui_btns, out])
display(ui)
